# Azure AI Agents with LangChain Integration

This notebook demonstrates how to use Azure AI Agent Service with LangChain through the `langchain-azure-ai` integration, powered by the Azure AI Projects SDK.

## Prerequisites

1. An Azure AI Foundry project
2. A deployed model (e.g., gpt-4o-mini)
3. Appropriate authentication credentials
4. Environment variables set:

   - `PROJECT_ENDPOINT`

5. Install `langchain-azure-ai`
   
   `pip install langchain-azure-ai[tools]` 

In [ ]:
import os

# Set up environment variables 
# note project connnection string is no longer supported
os.environ["AZURE_AI_PROJECT_ENDPOINT"] = "https://<resource>.services.ai.azure.com/api/projects/<project>"
os.environ["MODEL_DEPLOYMENT_NAME"] = "<deployment-name>"

## Setup

To start working with Azure AI Foundry Agents Service, you need to create an `AgentServiceFactory` which allows you to create agents in the service and materialize them in LangChain:

In [3]:
from langchain_azure_ai.agents import AgentServiceFactory
from azure.identity import DefaultAzureCredential

factory = AgentServiceFactory(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

## Basic Usage

The simplest way to interface with the Azure AI Foundry Agents service is to use the method `create_declarative_chat_agent` which creates a basic ReAct-like agent that has access to tools. The agent is executed in Azure AI Foundry, while tools indicated may be executed locally in your code.

In [ ]:
agent = factory.create_declarative_chat_agent(
    name="my-echo-agent",
    model="gpt-4.1",
    instructions="You are a helpful AI assistant that always replies back saying the opposite of what the user says.",
    trace=True
)

print(f"Agent with ID {factory.get_declarative_agents_id_from_graph(agent)} created.")

The agent has a single node as it doesn't contain any tool:

In [ ]:
from IPython import display

display.Image(agent.get_graph().draw_mermaid_png())

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="I'm a genius and I love programming!")]
state = agent.invoke({"messages": messages})

for m in state['messages']:
    m.pretty_print()

Agents created in Azure AI Foundry are objects that can be managed in the portal. If you want, you can delete the created agent in Azure AI Foundry:

In [ ]:
factory.delete_agent(agent)

Once the agent is deleted from Azure AI Foundry, the agent object in LangChain can't be used any longer.

## Advanced Configuration

You can customize the agent with various parameters. The Azure AI Agents service supports the following configuration options:

**Supported Parameters:**
- `temperature`: Controls randomness in the output (0.0 to 1.0)
- `top_p`: Controls diversity via nucleus sampling (0.0 to 1.0)  
- `response_format`: Specifies the format that the model must output
- `tools`: List of tools to provide to the agent
- `tool_resources`: Tool resources for the agent
- `metadata`: Custom metadata for the agent

**Note:** Some parameters like `max_completion_tokens`, `max_prompt_tokens`, etc. are not supported by the current Azure AI Agents SDK.

In [ ]:
# Create an agent with custom configuration
custom_agent = factory.create_declarative_chat_agent(
    name="custom-configured-agent",
    description="A creative writing assistant with poetic flair",
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    instructions="You are a creative writing assistant. Write in a poetic style.",
    temperature=0.8,  # Higher temperature for more creativity
)

# Test the custom agent
creative_response = custom_agent.invoke({ "messages": [HumanMessage("Describe a sunset over the ocean.")] })
print(f"Creative Response:\n{creative_response}")

## Tools

Agents can handle multiple type of tools.

### Callable tools

You can define callable tools that can be used with your agent. Tools are executed locally in your code.

Let's start by defining some tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]

Now, let's create an agent that can use those tools:

In [ ]:
math_agent = factory.create_declarative_chat_agent(
    name="math-agent",
    model="gpt-4.1",
    instructions="You are a helpful assistant tasked with performing arithmetic on a set of inputs.",
    tools=tools,
)

A node "tools" is added to the graph:

In [ ]:
from IPython import display

display.Image(math_agent.get_graph().draw_mermaid_png())

In [ ]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = math_agent.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

### Azure AI Services 

Azure AI Foundry have access to intelligent tools that can be used with your agents. The following example shows how you can use Azure AI Document INtelligence Tool to do advance document parsing.

In [ ]:
from langchain_azure_ai.tools import AzureAIDocumentIntelligenceTool

document_parser_agent = factory.create_declarative_chat_agent(
    name="document-agent",
    model="gpt-4.1",
    instructions="You are a helpful assistant tasked with analyzing documents.",
    tools=[AzureAIDocumentIntelligenceTool(credential=DefaultAzureCredential())],
)

Let's use the agent to parse the invoice:

In [ ]:
messages = [HumanMessage(content="What's the total amount in the invoice at https://raw.githubusercontent.com/Azure/azure-sdk-for-python/main/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/forms/Form_1.jpg")]
messages = document_parser_agent.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

### Azure AI Agent Service Tools

Azure AI Foundry Agent Service has specific tools available that can be executed directly within the service. For example, code interpreter. You can indicate those tools with your agent, although they may require configuration before using them:

In [17]:
import os
import tempfile
from pathlib import Path

# First, let's create some sample data for the code interpreter to work with
import pandas as pd

# Create sample sales data
data = {
    "month": ["Jan", "Feb", "Mar", "Apr", "May", "Jun"],
    "sales": [12000, 15000, 18000, 14000, 22000, 25000],
    "region": ["North", "South", "East", "West", "North", "South"],
}

df = pd.DataFrame(data)

# Create a CSV file in the current working directory
csv_path = Path.cwd() / "sample_sales_data.csv"
df.to_csv(
    csv_path, index=False, encoding="utf-8-sig"
)  # The code interpreter requires utf-8-sig

print(f"Created sample data file → {csv_path}")
print("\nSample data:")
print(df)

Created sample data file → /root/repos/santiagxf/langchain-azure/libs/azure-ai/docs/sample_sales_data.csv

Sample data:
  month  sales region
0   Jan  12000  North
1   Feb  15000  South
2   Mar  18000   East
3   Apr  14000   West
4   May  22000  North
5   Jun  25000  South


Use the projects SDK to configure your tools:

In [18]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (CodeInterpreterTool, FilePurpose,
                                    FileSearchTool, MessageRole)

client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

Upload the file for the code interpreter

In [24]:
uploaded_file = client.agents.files.upload_and_poll(
    file_path=str(csv_path),
    purpose=FilePurpose.AGENTS,  # Specify the purpose for the file
)
print(f"✓ Uploaded file, file ID: {uploaded_file.id}")

✓ Uploaded file, file ID: assistant-BGjtwoWCe7zbW2i9ssYT4a


Create the code interpreter tool:

In [25]:
code_interpreter = CodeInterpreterTool(file_ids=[uploaded_file.id])

Create the agent with access to this tool. Notice how `AgentServiceBaseTool` is used to tell LangChain that this particular tool needs to be executed in the cloud, instead of locally where the code is running:

In [26]:
from langchain_azure_ai.agents.prebuilt.tools import AgentServiceBaseTool
from langchain_core.messages import AIMessage, HumanMessage

code_interpreter_agent = factory.create_declarative_chat_agent(
    name="code-interpreter-agent",
    model="gpt-4.1",
    instructions="""You are a data analyst agent. 
                Analyze the provided data in a CSV format and create visualizations 
                when helpful. Use Python code to explore and understand the data.""",
    tools=[AgentServiceBaseTool(tool=code_interpreter)],
)

In [27]:
messages = [HumanMessage(content="create a pie chart with the data showing sales by region and show it to me as a png image.")]
messages = code_interpreter_agent.invoke({"messages": messages})

Found file path annotations: [{'type': 'file_path', 'text': 'sandbox:/mnt/data/sales_by_region.png', 'start_index': 111, 'end_index': 148, 'file_path': {'file_id': 'assistant-5sEKPZiGBtfLBnv7v57vex'}}]
Found file path annotation: file_path with text sandbox:/mnt/data/sales_by_region.png


In [28]:
for m in messages['messages']:
    m.pretty_print()
    if isinstance(m, AIMessage) and not isinstance(m.content, str):
        for block in m.content:
            if isinstance(block, dict) and block.get("type") == "image":
                image_data = block.get("base64")
                if image_data:
                    from IPython.display import Image, display
                    import base64
                    display(Image(data=base64.b64decode(image_data)))

================================ Human Message =================================

create a pie chart with the data showing sales by region and show it to me as a png image.
================================== Ai Message ==================================

I'll start by loading the data and inspecting it to identify the columns related to sales and region. Then, I'll create a pie chart displaying sales by region and save it as a PNG image.

Let's begin by examining the contents of your CSV file.
================================== Ai Message ==================================

The file contains the columns: `month`, `sales`, and `region`. To create a pie chart showing sales by region, I'll aggregate sales for each region and generate the chart.

I'll save the result as a PNG image and display it for you.
================================== Ai Message ==================================

The pie chart showing sales by region has been created. Here is the chart as a PNG image: 

![Sales by Re

## Async Operations

The service also supports async operations:

In [ ]:
import asyncio

from azure.identity import DefaultAzureCredential


async def async_example():
    # Create async agent
    async_agent = factory.create_declarative_chat_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Use model_name instead of model
        name="async-demo-agent",
        instructions="You are an async AI assistant.",
    )

    try:
        # Async generation
        response = await async_agent.ainvoke(
            { "messages": [HumanMessage("What are the benefits of asynchronous programming?")] }
        )
        print(f"Async Response: {response}")

    finally:
        # Clean up
        await factory.delete_agent(async_agent)

# Run the async example
await async_example()

### Other things to try

- Explore file search capabilities
- Implement custom function calling
- Build multi-tool agents
- Integrate with more complex LangChain workflows

The Azure AI Agents integration provides a powerful way to combine the capabilities of Azure AI Projects with the flexibility of LangChain, enabling you to build sophisticated AI applications with ease.